In [1]:
import os
import numpy as np
import SimpleITK
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
brainT1_path = 'BRATS-2/Image_Data/HG/0001/VSD.Brain.XX.O.MR_T1/VSD.Brain.XX.O.MR_T1.685.mha'
brainT1c_path ='BRATS-2/Image_Data/HG/0001/VSD.Brain.XX.O.MR_T1c/VSD.Brain.XX.O.MR_T1c.686.mha'
brainT2_path = 'BRATS-2/Image_Data/HG/0001/VSD.Brain.XX.O.MR_T2/VSD.Brain.XX.O.MR_T2.687.mha'
brainFLAIR_path = 'BRATS-2/Image_Data/HG/0001/VSD.Brain.XX.O.MR_Flair/VSD.Brain.XX.O.MR_Flair.684.mha'
brainT1_img = SimpleITK.ReadImage(brainT1_path)
brainT1c_img = SimpleITK.ReadImage(brainT1c_path)
brainT2_img = SimpleITK.ReadImage(brainT2_path)
brainFlair_img = SimpleITK.ReadImage(brainFLAIR_path)
truth_path = 'BRATS-2/Image_Data/HG/0001/VSD.Brain_3more.XX.XX.OT/VSD.Brain_3more.XX.XX.OT.6560.mha'
truth_img = SimpleITK.ReadImage(truth_path)
atlas_path = 'Atlas/mni_icbm152_t1_tal_nlin_sym_09a.nii'
atlas_img = SimpleITK.ReadImage(atlas_path)
initial_seg_path = 'BRATS-2/Image_Data/HG/0001/seg.nii.gz'
initial_seg = SimpleITK.ReadImage(initial_seg_path)

A method to display slices

In [3]:
def sitk_show(img, title=None, margin=0.0, dpi=40):
    nda = SimpleITK.GetArrayFromImage(img)
    #spacing = img.GetSpacing()
    figsize = (1 + margin) * nda.shape[0] / dpi, (1 + margin) * nda.shape[1] / dpi
    #extent = (0, nda.shape[1]*spacing[1], nda.shape[0]*spacing[0], 0)
    extent = (0, nda.shape[1], nda.shape[0], 0)
    fig = plt.figure(figsize=figsize, dpi=dpi)
    ax = fig.add_axes([margin, margin, 1 - 2*margin, 1 - 2*margin])
    plt.set_cmap("gray")
    ax.imshow(nda,extent=extent,interpolation=None)
    
    if title:
        plt.title(title)
    
    plt.show()

Takes an image segmentation (gray matter, white matter and CSF) and initializes theta. outliers are more than 3 standard deviations away from norm

In [84]:
def init_theta(seg,img, theta, k, c):
    for i in range(0,c):
        outliers = []
        for j in range(0,k):
            theta[i][2*j] = img[i][seg==(j+1)].mean()
            theta[i][2*j+1] = img[i][seg==(j+1)].std()
            outliers.extend(img[i][seg==(j+1)].tolist())
        theta[i][2*k] = np.array(outliers).mean()
        theta[i][2*k+1] = np.array(outliers).std()
            
    
        

In [5]:
k = 3 #3 tissue classes
c = 4 # my data has 4 channels
N = brainT1_img.GetHeight()*brainT1_img.GetWidth()*brainT1_img.GetDepth()
theta = np.empty([c, 2*(k+1)], double) #[c1k1,c1k2....cnkm, c1k+1, c2k+1...]
alpha = np.empty(N)

In [7]:
img = np.empty([c,N])
img[0] = SimpleITK.GetArrayFromImage(brainT1_img).flatten()
img[1] = SimpleITK.GetArrayFromImage(brainT1c_img).flatten()
img[2] = SimpleITK.GetArrayFromImage(brainT2_img).flatten()
img[3] = SimpleITK.GetArrayFromImage(brainFlair_img).flatten()
initial_seg = SimpleITK.GetArrayFromImage(initial_seg).flatten()

In [85]:
init_theta(initial_seg, img, theta, k, c)

In [86]:
theta

array([[ 215.75135229,  108.46821808,  361.07009317,   28.8918784 ,
         440.77736211,   28.5018692 ,  388.36491569,   88.39553045],
       [ 282.41242049,  217.17645632,  481.10535859,  127.54961949,
         569.68475254,   93.09325441,  506.69688688,  157.23965233],
       [ 554.45312432,  325.82081462,  492.32309043,  152.63318548,
         422.60372712,  170.37432576,  460.16613176,  198.5062879 ],
       [ 159.8251547 ,  160.15699176,  368.71675803,  104.46966392,
         415.60830657,  114.31180674,  369.0452824 ,  144.41501836]])